In [36]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, optimizers
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

import numpy as np
import pandas as pd

#For resizing the images 
import cv2,glob
from google.colab.patches import cv2_imshow

#For face detection and cropping
import pkg_resources

#For sorting the images in order
from tkinter import Tcl


## Loading the models and making predictions on unseen images

In [ ]:
# load the model for all the attributes
model_wrinkles = load_model("/content/drive/MyDrive/model_wrinkles1")
model_freakles = load_model("/content/drive/MyDrive/model_freakles4")
model_glasses = load_model("/content/drive/MyDrive/model_glasses")
model_haircolor = load_model("/content/drive/MyDrive/model_haircolor3")
model_hairtop = load_model("/content/drive/MyDrive/model_hairtop")

#Unseen Images
new_images = glob.glob('/content/drive/MyDrive/test_images/*jp*')

#Images sorted
new_images_sorted = Tcl().call('lsort', '-dict', new_images)
image_name = []
for image in new_images_sorted:
  image_name.append(image.split('/')[-1])

#Detecting and cropping face for wrinkles and freakles.
new_test_images_binary = []

for each_image in new_images:
  image = cv2.imread(each_image)   
  gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
  haar_xml = pkg_resources.resource_filename('cv2', 'data/haarcascade_frontalface_default.xml')
  face_cascade = cv2.CascadeClassifier(haar_xml)
  faces = face_cascade.detectMultiScale(gray, 1.25, 6)
  if(len(faces) == 1):
    for (x, y, w, h) in faces:
      cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)
      faces = image[y:y + h, x:x + w]
      faces = cv2.resize(faces, (200, 200))
      new_test_images_binary.append(faces)
  else:
    faces = cv2.resize(image, (200, 200))
    new_test_images_binary.append(faces) 

#Not detecting the face for multi class classification as hair will not be cropped
new_test_images_multi = []
for each_image in new_images:
  image = cv2.imread(each_image)
  image = cv2.resize(image, (200, 200))
  new_test_images_multi.append(image) 

df_binary = np.array(new_test_images_binary)
df_multi = np.array(new_test_images_multi)

#Making predictions
y_pred1 = model_wrinkles.predict(df_binary)
y_pred2 = model_freakles.predict(df_binary)
y_pred3 = model_glasses.predict(df_multi)
y_pred4 = model_haircolor.predict(df_multi)
y_pred5 = model_hairtop.predict(df_multi)

y_pred_wrinkles = np.argmax(y_pred1, axis=1)
y_pred_freakles = np.argmax(y_pred2, axis=1)
y_pred_glasses = np.argmax(y_pred3, axis=1)
y_pred_haircolor = np.argmax(y_pred4, axis=1)
y_pred_hairtop = np.argmax(y_pred5, axis=1)

#print(y_pred_wrinkles, y_pred_freakles, y_pred_glasses, y_pred_haircolor, y_pred_hairtop)

## Writing the predictions to a csv file

In [38]:
Result_df = pd.DataFrame()
Result_df['image_name'] = image_name
Result_df['wrinkles'] = y_pred_wrinkles
Result_df['freakles'] = y_pred_freakles
Result_df['glasses'] = y_pred_glasses
Result_df['haircolor'] = y_pred_haircolor
Result_df['hairtop'] = y_pred_hairtop

Result_df.to_csv('/content/drive/MyDrive/Results.csv', index=0)
